In [ ]:
import string
punc = string.punctuation[:2] + string.punctuation[3:]
print(punc)
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    return text 

In [ ]:
import re

def tokenization(text):
    text = re.split('\W+', text)
    return text


In [ ]:
import nltk
stopword = nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text

In [ ]:
ps = nltk.PorterStemmer()

def stemming(text):
    text = [ps.stem(word) for word in text]
    return text

In [ ]:
wn = nltk.WordNetLemmatizer()

def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return text

In [2]:
from __future__ import division
import os
from normalise.normalisation import normalise  # imports the normalisation module
import re

languages = ['us','es']
# to load the language specific train data
for language in languages:
   file = open(os.getcwd() + '/' + language + '_train.text', 'r')
   output_file = open(os.getcwd() + '/' + language + '_train_normal.text', 'w')
# string manipulation operations
   for line in file:
      line = re.sub(r'[\s]+\)', ')', re.sub(r'\([\s]+', '(', line)) # regex to remove a whitespace before and after a paranthesis
      line = re.sub(r'[\s]+\]', ']', re.sub(r'\[[\s]+', '[', line)) # regex to remove a whitespace before and after a square bracket
      line = re.sub(r'[\s]+\}', '}', re.sub(r'\{[\s]+', '{', line)) # regex to remove a whitespace before and after a flower bracket
      line = re.sub(r'(?i)\b((?:https?:\/\/|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}\/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '', line)   # regex to remove the urls and twitter links
      line = re.sub(r'-', '', line) # regex to remove '-'
      line = re.sub(r'(.)\1{9,}', '', line) #regex to remove repetitive characters
      z = normalise(line, verbose=False, variety="Ame")  # calling the normalise function, verbose mode: prints out complete run of program, variety: to specify as american english
      print(z, file=output_file) # writes the data to an output file

     |████████████████████████████████| 15.7 MB 1.6 MB/s 


ModuleNotFoundError: No module named 'sklearn.semi_supervised.label_propagation'

In [ ]:
import numpy
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
from tqdm import tqdm


def tokenize(text, lowercase=True):
    if lowercase:
        text = text.lower()
    return text.split()


def twitter_preprocess():
    preprocessor = TextPreProcessor(
        normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
                   'time',
                   'date', 'number'],
        annotate={"hashtag", "elongated", "allcaps", "repeated", 'emphasis',
                  'censored'},
        all_caps_tag="wrap",
        fix_text=True,
        segmenter="twitter_2018",
        corrector="twitter_2018",
        unpack_hashtags=True,
        unpack_contractions=True,
        spell_correct_elong=False,
        tokenizer=SocialTokenizer(lowercase=True).tokenize,
        dicts=[emoticons]
    ).pre_process_doc

    def preprocess(name, dataset):
        desc = "PreProcessing dataset {}...".format(name)

        data = [preprocessor(x)
                for x in tqdm(dataset, desc=desc)]
        return data

    return preprocess


def vectorize(sequence, el2idx, max_length, unk_policy="random",
              spell_corrector=None):
    """
    Covert array of tokens, to array of ids, with a fixed length
    and zero padding at the end
    Args:
        sequence (): a list of elements
        el2idx (): dictionary of word to ids
        max_length ():
        unk_policy (): how to handle OOV words
        spell_corrector (): if unk_policy = 'correct' then pass a callable
            which will try to apply spell correction to the OOV token


    Returns: list of ids with zero padding at the end

    """
    words = numpy.zeros(max_length).astype(int)

    # trim tokens after max length
    sequence = sequence[:max_length]

    for i, token in enumerate(sequence):
        if token in el2idx:
            words[i] = el2idx[token]
        else:
            if unk_policy == "random":
                words[i] = el2idx["<unk>"]
            elif unk_policy == "zero":
                words[i] = 0
            elif unk_policy == "correct":
                corrected = spell_corrector(token)
                if corrected in el2idx:
                    words[i] = el2idx[corrected]
                else:
                    words[i] = el2idx["<unk>"]

    return words